In [3]:
import itertools
import operator
import pickle
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split

from nodestimation.learning.connectoming import make_connectome
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL3 = [
    'N3S2',
    'S3R1'
]
ENGEL4 = [
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

AGE = {
    'B1C2': 28.0,
    'B1R1': 16.0,
    'G1R1': 23.0,
    'G1V2': 5.0,
    'J1T2': 8.0,
    'K1V1': 10.0,
    'K4L2': 14.0,
    'L1P1': 16.0,
    'L2M1': 20.0,
    'M1G2': 8.0,
    'M1N2': 7.0,
    'M2S2': 18.0,
    'N2K2': 30.0,
    'N3S2': 10.0,
    'O1O2': 18.0,
    'R1D2': 6.5,
    'P1H2': 7.0,
    'S1A2': 12.0,
    'S1B1': 17.0,
    'S1H1': 28.0,
    'S3R1': 19.0,
    'S1U3': 15.0,
}

SEX = {
    'B1C2': 'f',
    'B1R1': 'm',
    'G1R1': 'f',
    'G1V2': 'm',
    'J1T2': 'f',
    'K1V1': 'f',
    'K4L2': 'f',
    'L1P1': 'f',
    'L2M1': 'f',
    'M1G2': 'm',
    'M1N2': 'm',
    'M2S2': 'm',
    'N2K2': 'm',
    'N3S2': 'm',
    'O1O2': 'f',
    'R1D2': 'f',
    'P1H2': 'm',
    'S1A2': 'm',
    'S1B1': 'm',
    'S1H1': 'm',
    'S3R1': 'm',
    'S1U3': 'f',
}

SUBJECTS = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

CONNECTOMES_KIND = 'initial'

All computation has been already done, loading of the existing file with the solution...


In [7]:
subjects = SUBJECTS.copy()

DATASETS = {
    subject.name: {
        freq: {
            method: make_connectome(subject, freq, method, CONNECTOMES_KIND, threshold=1)
           for method in subject.connectomes[freq]
        }
        for freq in subject.connectomes
    }
    for subject in subjects if subject.name in [*ENGEL1, *ENGEL2]
}
print('Connectomes are prepared')

Connectomes are prepared


In [ ]:
GRAPHS = [
    metric_for_hemispheres(subjects, nx.algorithms.cluster.transitivity),
    metric_for_hemispheres(subjects, nx.algorithms.smetric.s_metric, normalized=False),
    metric_for_hemispheres(subjects, nx.algorithms.global_efficiency),
]
print('Graphs are prepared')

In [9]:
print('Processing centrality...')

datasets = DATASETS.copy()

ignored_methods = [
#     'wpli',
#     'envelope',
#     'coh',
#     'imcoh',
#     'plv',
#     'ciplv',
#     'ppc',
#     'pli',
#     'pli2_unbiased',
#     'wpli2_debiased'
]

close = dict()
between = dict()
eigen = dict()
degree = dict()
info = dict()
harmony = dict()
katz = dict()
percolation = dict()

total_ignored_subjects = 0

for subject_name in datasets:
    is_corrupted = False
    close.update({subject_name: dict()})
    between.update({subject_name: dict()})
    eigen.update({subject_name: dict()})
    degree.update({subject_name: dict()})
    info.update({subject_name: dict()})
    harmony.update({subject_name: dict()})
    katz.update({subject_name: dict()})
    percolation.update({subject_name: dict()})

    for freq in datasets[subject_name]:

        if is_corrupted:
            break

        for method in datasets[subject_name][freq]:

            if method in ignored_methods:
                continue

            print(subject_name, freq, method)
            label_names = datasets[subject_name][freq][method].columns

            if nx.is_connected(
                    nx.convert_matrix.from_numpy_array(
                        datasets[subject_name][freq][method].to_numpy()
                    )
            ):
                arr  = datasets[subject_name][freq][method].to_numpy()
            else:
                try:
                    corrupted_subject = [subject for subject in subjects if subject.name == subject_name][0]
                    subjects.remove(corrupted_subject)
                except IndexError:
                    pass
                total_ignored_subjects += 1
                print(f'Graph is not connected: {subject_name}, {freq}, {method}.\n'
                      f'This subject will be ignored.\n'
                      f'Total of ignored subjects: {total_ignored_subjects}\n'
                      f'Total of spared subjects: {len(subjects)}\n')
                is_corrupted = True
                break
                # raise ValueError(f'Graph is not connected: {subject_name}, {freq}, {method}')
            # if matrix has negative values, shift it to make minimal value at least 0
            if arr.min().min() < 0:
                print(f'\tShifting connectome for {method} at {freq}')
                for i in range(arr.shape[0]):
                    for j in range(arr.shape[1]):
                        if i != j: arr[i, j] -= arr.min().min()
            G = nx.convert_matrix.from_numpy_array(arr)
            mapping = {node: label_name for node, label_name in zip(G, label_names)}
            max = np.max(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            min = np.min(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            states = {node: (np.sum(row) - min)/(max - min) for node, row in zip(label_names, arr)}
            G_num = G.copy()
            G = nx.relabel_nodes(G, mapping)
            if CONNECTOMES_KIND == 'initial' or CONNECTOMES_KIND == 'inverse' or CONNECTOMES_KIND == 'spatial':
                G_sup = nx.convert_matrix.from_numpy_array(
                        lmd.suppress(
                            datasets[subject_name][freq][method],
                            trigger= arr.mean().mean(),
                            optimal=0
                        ).to_numpy()
                    )
                G_sup = nx.relabel_nodes(G_sup, mapping)
            else:
                G_sup = G
            for place, data in zip(
                [
                    close[subject_name],
                    between[subject_name],
                    eigen[subject_name],
                    info[subject_name],
                    degree[subject_name],
                    harmony[subject_name],
                    katz[subject_name],
                    percolation[subject_name]
                ],[
                        nx.closeness_centrality(G, distance='weight'),
                        nx.betweenness_centrality(G_sup, weight='weight'),
                        nx.eigenvector_centrality_numpy(G, weight='weight'),
                        nx.information_centrality(G, weight='weight'),
                        dict(G.degree(weight='weight')),
                        nx.harmonic_centrality(G, distance='weight'),
                        nx.katz_centrality_numpy(G, weight='weight'),
                        nx.algorithms.centrality.percolation_centrality(G_sup, weight='weight', states=states)
                    ]
            ):
                place.update({
                    freq + '_' + method: pd.Series(data)
                })

print('Centrality computations are done')

Processing centrality...
P1H2 4-8Hz wpli
P1H2 4-8Hz envelope
P1H2 4-8Hz coh
P1H2 4-8Hz imcoh
P1H2 4-8Hz plv
P1H2 4-8Hz ciplv
P1H2 4-8Hz ppc
P1H2 4-8Hz pli
P1H2 4-8Hz pli2_unbiased
P1H2 4-8Hz wpli2_debiased
M2S2 4-8Hz wpli
M2S2 4-8Hz envelope
M2S2 4-8Hz coh
M2S2 4-8Hz imcoh
M2S2 4-8Hz plv
M2S2 4-8Hz ciplv
M2S2 4-8Hz ppc
M2S2 4-8Hz pli
M2S2 4-8Hz pli2_unbiased
M2S2 4-8Hz wpli2_debiased
R1D2 4-8Hz wpli
R1D2 4-8Hz envelope
R1D2 4-8Hz coh
R1D2 4-8Hz imcoh
R1D2 4-8Hz plv
R1D2 4-8Hz ciplv
R1D2 4-8Hz ppc
R1D2 4-8Hz pli
R1D2 4-8Hz pli2_unbiased
R1D2 4-8Hz wpli2_debiased
S1A2 4-8Hz wpli
S1A2 4-8Hz envelope
S1A2 4-8Hz coh
S1A2 4-8Hz imcoh
S1A2 4-8Hz plv
S1A2 4-8Hz ciplv
S1A2 4-8Hz ppc
S1A2 4-8Hz pli
S1A2 4-8Hz pli2_unbiased
S1A2 4-8Hz wpli2_debiased
S1H1 4-8Hz wpli
S1H1 4-8Hz envelope
S1H1 4-8Hz coh
S1H1 4-8Hz imcoh
S1H1 4-8Hz plv
S1H1 4-8Hz ciplv
S1H1 4-8Hz ppc
S1H1 4-8Hz pli
S1H1 4-8Hz pli2_unbiased
S1H1 4-8Hz wpli2_debiased
K1V1 4-8Hz wpli
K1V1 4-8Hz envelope
K1V1 4-8Hz coh
K1V1 4-8Hz imcoh
K1

In [11]:
print('Updating connectomes...')

datasets_centrality = dict()

for subject in subjects:

    if subject.name not in [*ENGEL1, *ENGEL2]:
        continue

    datasets_centrality.update({subject.name: dict()})
    datasets_centrality[subject.name].update({
        'close': pd.DataFrame(close[subject.name]),
        'between': pd.DataFrame(between[subject.name]),
        'eigen': pd.DataFrame(eigen[subject.name]),
        'info': pd.DataFrame(info[subject.name]),
        'degree': pd.DataFrame(degree[subject.name]),
        'harmony': pd.DataFrame(harmony[subject.name]),
        'katz': pd.DataFrame(katz[subject.name]),
        'percolation': pd.DataFrame(percolation[subject.name])
    })
    true = subject.datasets['eigen']['resected']
    datasets_centrality[subject.name]['close'] =\
        datasets_centrality[subject.name]['close'].assign(resected=true)
    datasets_centrality[subject.name]['between'] =\
        datasets_centrality[subject.name]['between'].assign(resected=true)
    datasets_centrality[subject.name]['eigen'] =\
        datasets_centrality[subject.name]['eigen'].assign(resected=true)
    datasets_centrality[subject.name]['degree'] =\
        datasets_centrality[subject.name]['degree'].assign(resected=true)
    datasets_centrality[subject.name]['info'] =\
        datasets_centrality[subject.name]['info'].assign(resected=true)
    datasets_centrality[subject.name]['harmony'] =\
        datasets_centrality[subject.name]['harmony'].assign(resected=true)
    datasets_centrality[subject.name]['katz'] =\
        datasets_centrality[subject.name]['katz'].assign(resected=true)
    datasets_centrality[subject.name]['percolation'] =\
        datasets_centrality[subject.name]['percolation'].assign(resected=true)

Updating connectomes...


In [2]:
path = f'/home/user/Documents/{CONNECTOMES_KIND}_centrality_en1&2.pkl'
pickle.dump(
    datasets_centrality,
    open(
        path,
        'wb'
    )
)

NameError: name 'datasets_centrality' is not defined

In [3]:
path = f'/home/user//Documents/{CONNECTOMES_KIND}_centrality_en1&2.pkl'

datasets_centrality = pickle.load(open(path, 'rb'))

In [33]:
subjects_list = [*ENGEL1, *ENGEL2, *ENGEL3, *ENGEL4]

for subject in subjects:
    if subject.name not in subjects_list:
        continue
    subject.datasets = datasets_centrality[subject.name]

all_subjects = subjects.copy()

subjects = [subject for subject in subjects if subject.name in subjects_list]

print(f'Connectomes successfully updated. Now each subject has new {CONNECTOMES_KIND} connectomes')

print('Collecting statistics...')

stat1 = SubjectsStatistic(subjects, 'resected', centrality_metric='eigen')
print('Eigencentrality statistics are collected')
stat2 = SubjectsStatistic(subjects, 'resected', centrality_metric='between')
print('Betweenness centrality statistics are collected')
stat3 = SubjectsStatistic(subjects, 'resected', centrality_metric='close')
print('Closeness centrality statistics are collected')
stat4 = SubjectsStatistic(subjects, 'resected', centrality_metric='degree')
print('Degree centrality statistics are collected')
stat5 = SubjectsStatistic(subjects, 'resected', centrality_metric='info')
print('Info centrality statistics are collected')
stat6 = SubjectsStatistic(subjects, 'resected', centrality_metric='harmony')
print('Harmony centrality statistics are collected')
stat7 = SubjectsStatistic(subjects, 'resected', centrality_metric='katz')
print('Katz centrality statistics are collected')
stat8 = SubjectsStatistic(subjects, 'resected', centrality_metric='percolation')
print('Percolation centrality statistics are collected')
print(f'All statistics for {CONNECTOMES_KIND} connectomes are collected')
print('Saving statistics')

Connectomes successfully updated. Now each subject has new initial connectomes
Eigencentrality statistics are collected
Betweenness centrality statistics are collected
Closeness centrality statistics are collected
Degree centrality statistics are collected
Info centrality statistics are collected
Harmony centrality statistics are collected
Katz centrality statistics are collected
Percolation centrality statistics are collected
All statistics for initial connectomes are collected
Saving statistics


In [34]:
stats = (
        stat1,
        stat2,
        stat3,
        stat4,
        stat5,
        stat6,
        stat7,
        stat8,
    )

In [ ]:
path = f'/home/user//Documents/{CONNECTOMES_KIND}_stats_en1&2.pkl'
pickle.dump(
    (
        stat1,
        stat2,
        stat3,
        stat4,
        stat5,
        stat6,
        stat7,
        stat8,
    ),
    open(
        path,
        'wb'
    )
)

print(f'All statistics are successfully saved at {path = }')

In [4]:
path = f'/home/user//Documents/{CONNECTOMES_KIND}_stats_en1&2.pkl'
stats = pickle.load(open(path, 'rb'))

In [35]:
test1 = stats[0].test(state='reflected')
test2 = stats[1].test(state='reflected')
test3 = stats[2].test(state='reflected')
test4 = stats[3].test(state='reflected')
test5 = stats[4].test(state='reflected')
test6 = stats[5].test(state='reflected')
test7 = stats[6].test(state='reflected')
test8 = stats[7].test(state='reflected')
test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples, test7_samples, test8_samples = list(), list(), list(), list(), list(), list(), list(), list()
for feature in test1.result:
    test1_samples.append(test1.result[feature][1])
    test2_samples.append(test2.result[feature][1])
    test3_samples.append(test3.result[feature][1])
    test4_samples.append(test4.result[feature][1])
    test5_samples.append(test5.result[feature][1])
    test6_samples.append(test6.result[feature][1])
    test7_samples.append(test7.result[feature][1])
    test8_samples.append(test8.result[feature][1])

test_samples = np.array([
    np.array(test1_samples),
    np.array(test2_samples),
    np.array(test3_samples),
    np.array(test4_samples),
    np.array(test5_samples),
    np.array(test6_samples),
    np.array(test7_samples),
    np.array(test8_samples)
])

df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
    'eigen', 'between', 'close', 'degree', 'info', 'harmony', 'katz', 'percolation'
]).T
print(df)
df.to_csv('/home/user/Documents/Wilcoxon_ENG_1&2&3&4.csv')

                             eigen   between     close    degree      info  \
4-8Hz_wpli            6.128071e-01  0.515587  0.568135  0.654712  0.623172   
4-8Hz_envelope        2.935385e-01  0.708642  0.264799  0.275017  0.284175   
4-8Hz_coh             1.553784e-07  0.077248  0.308666  0.001659  0.000857   
4-8Hz_imcoh           7.240341e-01  0.297239  0.013678  0.855252  0.811630   
4-8Hz_plv             4.961603e-06  0.346741  0.502324  0.001578  0.001327   
4-8Hz_ciplv           6.825575e-01  0.921600  0.160479  0.739529  0.708651   
4-8Hz_ppc             1.668446e-08  0.151427  0.301727  0.000238  0.000146   
4-8Hz_pli             6.825575e-01  0.304230  0.995305  0.802515  0.779849   
4-8Hz_pli2_unbiased   5.385979e-01  0.662161  0.312879  0.741752  0.710842   
4-8Hz_wpli2_debiased  9.320023e-01  0.890292  0.000069  0.823059  0.818483   

                       harmony          katz  percolation  
4-8Hz_wpli            0.544443  6.483526e-01     0.375820  
4-8Hz_envelope       

In [36]:
test1 = stats[0].test(state='reflected', test='mannwhitneyu')
test2 = stats[1].test(state='reflected', test='mannwhitneyu')
test3 = stats[2].test(state='reflected', test='mannwhitneyu')
test4 = stats[3].test(state='reflected', test='mannwhitneyu')
test5 = stats[4].test(state='reflected', test='mannwhitneyu')
test6 = stats[5].test(state='reflected', test='mannwhitneyu')
test7 = stats[6].test(state='reflected', test='mannwhitneyu')
test8 = stats[7].test(state='reflected', test='mannwhitneyu')
test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples, test7_samples, test8_samples = list(), list(), list(), list(), list(), list(), list(), list()
for feature in test1.result:
    test1_samples.append(test1.result[feature][1])
    test2_samples.append(test2.result[feature][1])
    test3_samples.append(test3.result[feature][1])
    test4_samples.append(test4.result[feature][1])
    test5_samples.append(test5.result[feature][1])
    test6_samples.append(test6.result[feature][1])
    test7_samples.append(test7.result[feature][1])
    test8_samples.append(test8.result[feature][1])

test_samples = np.array([
    np.array(test1_samples),
    np.array(test2_samples),
    np.array(test3_samples),
    np.array(test4_samples),
    np.array(test5_samples),
    np.array(test6_samples),
    np.array(test7_samples),
    np.array(test8_samples)
])

df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
    'eigen', 'between', 'close', 'degree', 'info', 'harmony', 'katz', 'percolation'
]).T
print(df)
df.to_csv('/home/user/Documents/MannWitneyu_ENG_1&2&3&4.csv')

                         eigen   between     close    degree      info  \
4-8Hz_wpli            0.347972  0.356870  0.260572  0.357286  0.356506   
4-8Hz_envelope        0.272239  0.419249  0.450934  0.447626  0.463368   
4-8Hz_coh             0.000010  0.124518  0.445148  0.005077  0.006110   
4-8Hz_imcoh           0.394542  0.120871  0.067556  0.282038  0.341046   
4-8Hz_plv             0.000350  0.294859  0.339514  0.025104  0.024377   
4-8Hz_ciplv           0.494998  0.475000  0.445974  0.424585  0.347200   
4-8Hz_ppc             0.000002  0.196611  0.285578  0.012400  0.011873   
4-8Hz_pli             0.500000  0.162734  0.368653  0.479999  0.500000   
4-8Hz_pli2_unbiased   0.383325  0.365107  0.153405  0.450521  0.496665   
4-8Hz_wpli2_debiased  0.434433  0.420078  0.001188  0.186659  0.138580   

                       harmony          katz  percolation  
4-8Hz_wpli            0.248505  3.311359e-01     0.249831  
4-8Hz_envelope        0.475837  2.612507e-01     0.456731  
4-8Hz

In [127]:
i = 0
true = stats[i].datasets['true']
false = stats[i].datasets['false_mirror']
scaler = MaxAbsScaler()
print(stats[i].centrality_metric)


true = pd.DataFrame(scaler.fit_transform(true), columns=true.columns)
false = pd.DataFrame(scaler.fit_transform(false), columns=false.columns)

for feat in true.columns:
    w, p = sp.stats.ttest_ind(true[feat], false[feat])
    print(feat[6:], p)


feat = 'wpli'
# for val1, val2 in zip(stats[0].datasets['true']['4-8Hz_coh'], stats[0].datasets['false_res']['4-8Hz_coh']):
#     print(val1, val2)

# print(
#     np.mean(
#         np.array([
#             np.abs(val1 - val2)
#             for val1, val2 in zip(stats[i].datasets['true'][f'4-8Hz_{feat}'],
#                                   stats[i].datasets['false_res'][f'4-8Hz_{feat}'])
#         ])
#     )
# )

# x = np.arange(stats[i].datasets['true'][f'4-8Hz_{feat}'].to_numpy().shape[0])
#
# plt.plot(
#     x, stats[i].datasets['true'][f'4-8Hz_{feat}'].to_numpy(), 'gx',
#     x, stats[i].datasets['false_mirror'][f'4-8Hz_{feat}'].to_numpy(), 'ro'
# )
# plt.show()
#
# feat = 'envelope'
#
# plt.plot(
#     x, stats[i].datasets['true'][f'4-8Hz_{feat}'].to_numpy(), 'gx',
#     x, stats[i].datasets['false_mirror'][f'4-8Hz_{feat}'].to_numpy(), 'ro'
# )
# plt.show()

eigen
wpli 0.0008117911888701784
envelope 0.34944988965725554
coh 0.07040544183835014
imcoh 2.535107137548402e-06
plv 0.3148843146982729
ciplv 0.04072622824960368
ppc 0.23778742919111537
pli 0.4007241200601771
pli2_unbiased 1.2199289786234054e-05
wpli2_debiased 9.457825168762198e-24


In [161]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression

i = 7
true = stats[i].datasets['true'].assign(resected=True)
false = stats[i].datasets['false_mirror'].assign(resected=False)
dataset = pd.concat([true, false], axis=0).sample(frac=1)
resected = dataset['resected']
dataset = dataset.drop(columns='resected')
model = LinearRegression()
scaler = StandardScaler()
# dataset = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)
# dataset = dataset.assign(resected=resected.to_list())
X = scaler.fit_transform(dataset)
y = resected.to_numpy()
model.fit(X, y)
importance = model.coef_
print('--------------')
print('LinearRegression importance')
for i, v in zip(dataset.columns, importance):
	print(f'Feature: {i}, Score: {v:.5f}')
print('--------------')
model = LogisticRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_[0]
print('--------------')
print('LogisticRegression importance')
for i, v in zip(dataset.columns, importance):
	print(f'Feature: {i}, Score: {v:.5f}')
print('--------------')
model = RandomForestClassifier()
# fit the model
model.fit(X, y)
# get importance
importance = model.feature_importances_
print('--------------')
print('RandomForestClassifier importance')
for i, v in zip(dataset.columns, importance):
	print(f'Feature: {i}, Score: {v:.5f}')
print('--------------')

model = KNeighborsClassifier()
# fit the model
model.fit(X, y)
# perform permutation importance
results = permutation_importance(model, X, y, scoring='accuracy')
# get importance
importance = results.importances_mean
print('--------------')
print('permutation importance')
for i, v in zip(dataset.columns, importance):
	print(f'Feature: {i}, Score: {v:.5f}')
print('--------------')

model = SVC(kernel='linear')
model.fit(X, y)
importance = model.coef_
print('--------------')
print('Linear SVM importance')
for i, v in zip(dataset.columns, *importance):
	print(f'Feature: {i}, Score: {v:.5f}')
print('--------------')




--------------
LinearRegression importance
Feature: 4-8Hz_wpli, Score: -0.03004
Feature: 4-8Hz_envelope, Score: -0.01351
Feature: 4-8Hz_coh, Score: -0.10160
Feature: 4-8Hz_imcoh, Score: 0.01274
Feature: 4-8Hz_plv, Score: 0.01002
Feature: 4-8Hz_ciplv, Score: 0.03342
Feature: 4-8Hz_ppc, Score: 0.04246
Feature: 4-8Hz_pli, Score: 0.04595
Feature: 4-8Hz_pli2_unbiased, Score: -0.02892
Feature: 4-8Hz_wpli2_debiased, Score: 0.01127
--------------
--------------
LogisticRegression importance
Feature: 4-8Hz_wpli, Score: -0.12050
Feature: 4-8Hz_envelope, Score: -0.05405
Feature: 4-8Hz_coh, Score: -0.40104
Feature: 4-8Hz_imcoh, Score: 0.05214
Feature: 4-8Hz_plv, Score: 0.02765
Feature: 4-8Hz_ciplv, Score: 0.13471
Feature: 4-8Hz_ppc, Score: 0.16557
Feature: 4-8Hz_pli, Score: 0.18766
Feature: 4-8Hz_pli2_unbiased, Score: -0.11541
Feature: 4-8Hz_wpli2_debiased, Score: 0.04577
--------------
--------------
RandomForestClassifier importance
Feature: 4-8Hz_wpli, Score: 0.10561
Feature: 4-8Hz_envelope, Sc